In [1]:
from variable.variable import Variable
from activation_functions.relu import ReLUActivation
relu_fn = ReLUActivation()

In [2]:
x = Variable(3)
y = Variable(4)

In [3]:
z = x + y 

In [4]:
from models.simple_model import SimpleModel
input_size = 3
hidden_size = 10
output_size = 1


model = SimpleModel(input_size, hidden_size, output_size)

In [5]:
from optimizer.sgd import SGDOptimizer
optimizer = SGDOptimizer(model.parameters(), learning_rate=0.01)


x = [[Variable(1), Variable(2), Variable(3)]]

output = model(x)


In [6]:
output[0][0]

Variable(data=16.845228957484792, grad=0), _op=<ops.basic_ops.AddOperation object at 0x000002300AB12A50>, _prev={Variable(data=16.49250871577417, grad=0), _op=<ops.basic_ops.AddOperation object at 0x000002300AB12960>, _prev={Variable(data=2.130102163630818, grad=0), _op=<ops.basic_ops.MulOperation object at 0x000002300AB12870>, _prev={Variable(data=3.4698393022387006, grad=0), _op=<activation_functions.relu.ReLUActivation object at 0x000002300AA44FE0>, _prev={Variable(data=3.4698393022387006, grad=0), _op=<ops.basic_ops.AddOperation object at 0x000002300AB110D0>, _prev={Variable(data=2.8120374623234907, grad=0), _op=<ops.basic_ops.AddOperation object at 0x000002300AB10FE0>, _prev={Variable(data=2.1145528119333643, grad=0), _op=<ops.basic_ops.MulOperation object at 0x000002300AB10EF0>, _prev={Variable(data=3, grad=0), _op=None, _prev=set(), _name=None, Variable(data=0.7048509373111215, grad=0), _op=None, _prev=set(), _name=weight_2_9}, _name=None, Variable(data=0.6974846503901265, grad=

In [7]:
from loss_functions.mse import MSELoss
from variable.variable import Variable
loss_fn = MSELoss()

target = [[Variable(2)]]
loss = loss_fn(output, target)


In [10]:
loss_fn.backward()

AttributeError: 'MSELoss' object has no attribute 'backwards'

In [9]:
model.layer1.weight

[[Variable(data=0.7062873974296746, grad=5.546320427261335), _op=None, _prev=set(), _name=weight_0_0,
  Variable(data=0.20634341552299018, grad=11.09264085452267), _op=None, _prev=set(), _name=weight_1_0,
  Variable(data=0.7281460899453482, grad=16.638961281784006), _op=None, _prev=set(), _name=weight_2_0],
 [Variable(data=0.43724008196859243, grad=1.5812971218813787), _op=None, _prev=set(), _name=weight_0_1,
  Variable(data=0.7331721718684626, grad=3.1625942437627574), _op=None, _prev=set(), _name=weight_1_1,
  Variable(data=0.10937273568924444, grad=4.743891365644136), _op=None, _prev=set(), _name=weight_2_1],
 [Variable(data=0.5802114515851918, grad=1.8596711430384438), _op=None, _prev=set(), _name=weight_0_2,
  Variable(data=0.43314358626383864, grad=3.7193422860768877), _op=None, _prev=set(), _name=weight_1_2,
  Variable(data=0.49592629761143137, grad=5.5790134291153315), _op=None, _prev=set(), _name=weight_2_2],
 [Variable(data=0.4097319345556446, grad=9.375586574251747), _op=Non

In [10]:
optimizer.step()

In [11]:
model.layer1.weight

[[Variable(data=0.6508241931570613, grad=5.546320427261335), _op=None, _prev=set(), _name=weight_0_0,
  Variable(data=0.09541700697776347, grad=11.09264085452267), _op=None, _prev=set(), _name=weight_1_0,
  Variable(data=0.5617564771275082, grad=16.638961281784006), _op=None, _prev=set(), _name=weight_2_0],
 [Variable(data=0.4214271107497786, grad=1.5812971218813787), _op=None, _prev=set(), _name=weight_0_1,
  Variable(data=0.701546229430835, grad=3.1625942437627574), _op=None, _prev=set(), _name=weight_1_1,
  Variable(data=0.06193382203280308, grad=4.743891365644136), _op=None, _prev=set(), _name=weight_2_1],
 [Variable(data=0.5616147401548074, grad=1.8596711430384438), _op=None, _prev=set(), _name=weight_0_2,
  Variable(data=0.39595016340306977, grad=3.7193422860768877), _op=None, _prev=set(), _name=weight_1_2,
  Variable(data=0.44013616332027805, grad=5.5790134291153315), _op=None, _prev=set(), _name=weight_2_2],
 [Variable(data=0.3159760688131271, grad=9.375586574251747), _op=None,

In [16]:
optimizer.zero_grad()

In [17]:
model.layer1.weight

[[Variable(data=0.6508241931570613, grad=0), _op=None, _prev=set(), _name=weight_0_0,
  Variable(data=0.09541700697776347, grad=0), _op=None, _prev=set(), _name=weight_1_0,
  Variable(data=0.5617564771275082, grad=0), _op=None, _prev=set(), _name=weight_2_0],
 [Variable(data=0.4214271107497786, grad=0), _op=None, _prev=set(), _name=weight_0_1,
  Variable(data=0.701546229430835, grad=0), _op=None, _prev=set(), _name=weight_1_1,
  Variable(data=0.06193382203280308, grad=0), _op=None, _prev=set(), _name=weight_2_1],
 [Variable(data=0.5616147401548074, grad=0), _op=None, _prev=set(), _name=weight_0_2,
  Variable(data=0.39595016340306977, grad=0), _op=None, _prev=set(), _name=weight_1_2,
  Variable(data=0.44013616332027805, grad=0), _op=None, _prev=set(), _name=weight_2_2],
 [Variable(data=0.3159760688131271, grad=0), _op=None, _prev=set(), _name=weight_0_3,
  Variable(data=0.6776757602178587, grad=0), _op=None, _prev=set(), _name=weight_1_3,
  Variable(data=-0.27279583956273684, grad=0), _